In [1]:
# DON'T FORGET TO ACTIVATE THE GPU when on google colab (Edit > Notebook settings)

from os import environ
GOOGLE_COLAB = True if "COLAB_GPU" in environ else False
if GOOGLE_COLAB:
    !pip install swyft

In [2]:
import numpy as np
import pylab as plt
import swyft
import torch
from scipy import stats
%load_ext autoreload
%autoreload 2

np.random.seed(27)
torch.manual_seed(27)

In [4]:
#DEVICE = swyft.set_device(gpu=True)
DEVICE = 'cuda'

In [6]:
def model(params):
    a = params['a']
    b = params['b']
    x=np.array([a,2*(b-a)])
    return dict(x=x)

def noise(obs, params, noise = 0.01):
    x = obs['x']
    n = np.random.randn(*x.shape)*noise
    return dict(x=x + n)

par0 = dict(a=0.55, b=0.45)
obs0 = model(par0)

In [7]:
cache = swyft.MemoryCache(params = ["a", "b"], obs_shapes = {"x": (2,)})

Creating new cache.


In [8]:
prior = swyft.Prior({"a": ["uniform", 0., 1.], "b": ["uniform",  0., 1.]})

In [ ]:
s = swyft.SWYFT(model, noise, prior, cache, obs0, device = DEVICE)
s.infer1d(Ninit = 500)

Simulate:   8%|▊         | 38/495 [00:00<00:01, 379.69it/s]

N = 500
Round: 1
Adding 495 new samples. Run simulator!


Simulate: 100%|██████████| 495/495 [00:01<00:00, 386.27it/s]


n_features = 2
Start training
LR iteration 0
Validation loss: 2.7580266197522483
Validation loss: 1.9654800395170848
Validation loss: 1.566274379690488
Validation loss: 1.3233900517225266
Validation loss: 1.2979619776209195
Validation loss: 1.257373869419098
Validation loss: 1.271568976342678
Total epochs: 7
LR iteration 1
Validation loss: 1.2275660745799541
Validation loss: 1.2171886215607326
Validation loss: 1.1910312610367935
Validation loss: 1.1893476744492848
Validation loss: 1.1785331641634305
Validation loss: 1.187134472032388
Total epochs: 6


Simulate:   0%|          | 0/310 [00:00<?, ?it/s]

...

n_features = 2
Start training
LR iteration 0
Validation loss: 1.5824195061411177
Validation loss: 1.2455446319920676
Validation loss: 1.1111796685627529
Validation loss: 1.070530982528414
Validation loss: 1.147727634225573
Total epochs: 5
LR iteration 1
Validation loss: 1.066651165485382
Validation loss: 1.0315782078674862
Validation loss: 1.0251764527388982
Validation loss: 1.03256014415196
Total epochs: 4
Volume shrinkage: 0.7118877743879293
N = 1952
Round: 6


Simulate:   0%|          | 0/944 [00:00<?, ?it/s]

Adding 944 new samples. Run simulator!


Simulate: 100%|██████████| 944/944 [00:01<00:00, 482.55it/s]


n_features = 2
Start training
LR iteration 0
Validation loss: 1.3974789381027222
Validation loss: 1.2713766420880954
Validation loss: 1.2027178071439266
Validation loss: 1.212130518630147
Total epochs: 4
LR iteration 1


In [ ]:
post = s.posteriors()
swyft.corner(post, ["a", "b"], color='r', figsize = (15,15), truth=par0)

In [1]:
s.infer2d()

NameError: name 's' is not defined

In [ ]:
post = s.posteriors()
swyft.corner(post, ["a", "b"], color='r', figsize = (15,15), truth=par0)